In [20]:
import pandas as pd
import preprocessor as p
import contractions
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [22]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to C:\Users\hp
[nltk_data]     user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\hp
[nltk_data]     user\AppData\Roaming\nltk_data...


True

Read CSV file

In [2]:
data = pd.read_csv("D:\Skill Project\demo sentiment data.csv", encoding="ISO-8859-1")
data.columns = ["target","ids","date","flag","user","text"]
data.head()

,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [7]:
data.tail()

,target,ids,date,flag,user,text
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,1,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [3]:
data = pd.read_csv("D:\Skill Project\demo sentiment data.csv", names=data.columns, encoding="ISO-8859-1")
data.shape

(1600000, 6)

Checking Null Values

In [4]:
data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

Counting target(+ve or -ve)

In [5]:
data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Setting +ve Target to 1

In [6]:
data.replace({'target':{4:1}}, inplace=True)
data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

Stemming

In [9]:
def process_twts(twts):
    # applying tweet cleaning, then lowercasing and finally resolving contractions
    # removing special characters
    cleaned_twts = twts.map(lambda txt: ''.join(word if word.isalpha() or word.isspace() else ' ' for word in contractions.fix(p.clean(txt).lower())))
    # Tokenizing each sentences
    tokenized_twts = cleaned_twts.map(word_tokenize)
    # Removing Stop Words from sentences
    # Lemmatizing words in text
    stop_words = stopwords.words("english")
    lemmatizer = WordNetLemmatizer()
    lemmatiized_twts = tokenized_twts.map(lambda txt: [lemmatizer.lemmatize(word) for word in txt if word not in stop_words])

    return lemmatiized_twts

In [12]:
# Dividing the input text data and the associated output sentiment vector
twts, y = data["text"][600000: 1000000], data["target"][600000: 1000000]

In [23]:
# Processing all tweets by tokenizing, removing Stopwords and Lemmatizing words
processed_twts = process_twts(twts)

In [28]:
i = 700000
print("Raw: ", twts[i])
print("Processed: ", processed_twts[i])

Raw:  People are tweeting advice on how to combat tear gas in Iran.    #iranelection
Processed:  ['people', 'tweeting', 'advice', 'combat', 'tear', 'gas', 'iran']


In [29]:
x_train, x_test, y_train, y_test = train_test_split(processed_twts, y, test_size=0.2, stratify=y, random_state=5)
print(x_train.shape, x_test.shape)

(320000,) (80000,)


In [30]:
# This is just to pass by the tokenizing in TfidfVectorizer class
def tokeniz(token):
    return token

vectorizer_uni = TfidfVectorizer(tokenizer=tokeniz, lowercase=False)
vectorizer_uni_bi = TfidfVectorizer(tokenizer=tokeniz, ngram_range=(1, 2), lowercase=False)

x_train_vectorized_uni = vectorizer_uni.fit_transform(x_train)
x_train_vectorized_uni_bi = vectorizer_uni_bi.fit_transform(x_train)
x_test_vectorized_uni = vectorizer_uni.transform(x_test)
x_test_vectorized_uni_bi = vectorizer_uni_bi.transform(x_test)

c:\Python311\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [31]:
print(x_train_vectorized_uni[:5].toarray())
print(x_train_vectorized_uni.shape)
print(vectorizer_uni.get_feature_names_out())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(320000, 94075)
['aa' 'aaa' 'aaaa' ... 'zzzzzzzzzzzzzzzzzz'
 'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz'
 'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz']


In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
logRegModel_uni = LogisticRegression(max_iter=1000)
logRegModel_uni_bi = LogisticRegression(max_iter=1000)
NBModel_uni = GaussianNB()
NBModel_uni_bi = GaussianNB()

In [33]:
logRegModel_uni.fit(x_train_vectorized_uni, y_train)
logRegModel_uni_bi.fit(x_train_vectorized_uni_bi, y_train)

LogisticRegression(max_iter=1000)